In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
import ast
%config InlineBackend.figure_format = 'retina'
plt.rcParams['font.sans-serif'] = 'SimHei'
# 用于正常显示符号
plt.rcParams['axes.unicode_minus'] = False

In [4]:
df=pd.read_csv('./data/task2_dataset.csv')

In [5]:
df.head()

,uid,d,t,x,y
0,0,0,0,163,60
1,0,0,1,163,60
2,0,0,2,163,61
3,0,0,5,163,60
4,0,0,8,163,61


In [6]:
print(len(df))

29389749


In [7]:
df=df.drop(df[df['x']==999].index)

In [8]:
df['p']=(df['x']-1)*200+df['y']-1

In [9]:
df.head()

,uid,d,t,x,y,p
0,0,0,0,163,60,32459
1,0,0,1,163,60,32459
2,0,0,2,163,61,32460
3,0,0,5,163,60,32459
4,0,0,8,163,61,32460


In [10]:
df_train=df[df['d']<60]
df_valid=df[df['d']>=60]

In [11]:
df_train=df_train[['uid','p']]
df_valid=df_valid[['uid','p']]

In [12]:
data=df_train.values.tolist()

In [13]:
map_user1=[]
map_user2=[]
user_num=np.zeros((max(df['uid']))+1,dtype=np.int32)
for i in range((max(df['uid']))+1):
    map_user1.append(dict())
    map_user2.append(dict())

In [14]:
for i in range(len(data)):
    uid=data[i][0]
    p=data[i][1]
    if p not in map_user1[uid].keys():
        map_user1[uid][p]=user_num[uid]
        map_user2[uid][user_num[uid]]=p
        user_num[uid]+=1

In [15]:
cnt=[]
for i in range(max(df_train['uid'])+1):
    cnt.append(np.zeros((user_num[i],user_num[i])))

In [16]:
for i in range(len(data)):
    if i>0 and data[i][0]==data[i-1][0]:
        uid=data[i][0]
        pre=data[i-1][1]
        ne=data[i][1]
        cnt[uid][map_user1[uid][pre]][map_user1[uid][ne]]+=1

In [17]:
max_num_cnt=[]
for i in range(max(df_train['uid'])+1):
    max_num_cnt.append([])
for i in range(max(df_train['uid'])+1):
    for j in range(user_num[i]):
        max_num_cnt[i].append(np.argmax(cnt[i][j]))

In [18]:
last_p=[]
for i in range(len(data)):
    if i>0 and data[i][0]!=data[i-1][0]:
        last_p.append(map_user1[data[i-1][0]][data[i-1][1]])
    if i==len(data)-1:
        last_p.append(map_user1[data[i][0]][data[i][1]])

In [19]:
print(max_num_cnt[0])

[1, 1, 3, 4, 5, 6, 0, 0, 9, 9, 0, 12, 12, 12, 15, 9, 17, 1, 12, 20, 12, 9, 9, 1, 12, 24, 25, 28, 1, 12, 12, 24, 33, 34, 35, 36, 37, 35, 39, 34, 41, 42, 0, 1, 16, 9, 0, 0, 9, 50, 50, 46, 53, 12, 13, 12, 57, 58, 59, 1, 61, 62, 63, 64, 65, 64, 28, 12, 46, 12, 12, 20, 73, 74, 75, 0, 2, 13, 20, 11, 1, 13, 83, 22]


In [20]:
predict_num=np.zeros(max(df_valid['uid'])+1,dtype=np.int32)
data_valid=df_valid.values.tolist()

In [21]:
for i in range(len(data_valid)):
    predict_num[data_valid[i][0]]+=1

In [22]:
predict_p=[]
for i in range(max(df_valid['uid'])+1):
    for j in range(predict_num[i]):
        pre_point=max_num_cnt[i][last_p[i]]
        last_p[i]=pre_point
        predict_p.append(map_user2[i][pre_point])

In [23]:
df_valid=df[df['d']>=60]

In [24]:
df_valid['pred_p']=predict_p

/root/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
df_valid['pred_x']=df_valid['pred_p']//200+1
df_valid['pred_y']=df_valid['pred_p']%200+1

/root/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/root/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
df_valid.head()

,uid,d,t,x,y,p,pred_p,pred_x,pred_y
843,0,60,2,65,175,12974,32460,163,61
844,0,60,3,53,181,10580,32460,163,61
845,0,60,9,75,158,14957,32460,163,61
846,0,60,10,127,84,25283,32460,163,61
847,0,60,11,132,82,26281,32460,163,61


In [51]:
def calc_geobleu_t(df):
    reference = df[["d","t","x","y"]].to_numpy()
    generated = df[["d","t","pred_x","pred_y"]].to_numpy()
    return geobleu.calc_geobleu_single(generated.tolist(), reference.tolist()) #日付間の並列化不要。ユーザーごとに並列化する。

In [53]:
import geobleu
from joblib import Parallel, delayed
from tqdm import tqdm
result=[]
grouped = df_valid[["uid", "d", "t", "x", "y", "pred_x", "pred_y"]].groupby("uid")
results = Parallel(n_jobs=-1)(delayed(calc_geobleu_t)(df) for _, df in tqdm(grouped))

100%|██████████| 22500/22500 [05:51<00:00, 64.07it/s] 


In [56]:
print(sum(results)/len(results))

0.16067659089744382


In [60]:
def calc_dtw_t(df):
    reference = df[["d","t","x","y"]].to_numpy()
    generated = df[["d","t","pred_x","pred_y"]].to_numpy()
    return geobleu.calc_dtw_single(generated.tolist(), reference.tolist()) #日付間の並列化不要。ユーザーごとに並列化する。

In [61]:
import geobleu
from joblib import Parallel, delayed
from tqdm import tqdm
result=[]
grouped = df_valid[["uid", "d", "t", "x", "y", "pred_x", "pred_y"]].groupby("uid")
results = Parallel(n_jobs=-1)(delayed(calc_dtw_t)(df) for _, df in tqdm(grouped))



  0%|          | 0/22500 [00:00<?, ?it/s]

  0%|          | 1/22500 [00:12<77:11:08, 12.35s/it]

  0%|          | 12/22500 [00:20<8:56:49,  1.43s/it]

  0%|          | 24/22500 [00:20<3:46:45,  1.65it/s]

  0%|          | 36/22500 [00:21<2:04:13,  3.01it/s]

  0%|          | 60/22500 [00:21<54:55,  6.81it/s]  

  0%|          | 96/22500 [00:21<25:22, 14.71it/s]

  1%|          | 168/22500 [00:21<10:28, 35.51it/s]

  1%|          | 216/22500 [00:21<07:04, 52.49it/s]

  1%|          | 264/22500 [00:21<05:21, 69.25it/s]

  1%|▏         | 312/22500 [00:22<04:09, 88.88it/s]

  2%|▏         | 360/22500 [00:22<03:18, 111.62it/s]

  2%|▏         | 408/22500 [00:22<02:47, 131.63it/s]

  2%|▏         | 456/22500 [00:22<02:23, 153.69it/s]

  2%|▏         | 504/22500 [00:23<02:12, 165.75it/s]

  2%|▏         | 552/22500 [00:23<02:01, 180.40it/s]

  3%|▎         | 600/22500 [00:23<02:02, 179.32it/s]

  3%|▎         | 648/22500 [00:23<01:56, 187.84it/s]

  3%|▎         | 696/22500 [00:23<01:49, 19

In [62]:
print(sum(results)/len(results))

31.288469743136766


In [33]:
import geobleu

# tuple format: (d, t, x, y)
generated = [
    (60, 12, 84, 88),
    (60, 15, 114, 78),
    (60, 21, 121, 96),
    (61, 12, 78, 86),
    (61, 13, 89, 67),
    (61, 17, 97, 70),
    (61, 20, 96, 70),
    (61, 24, 111, 80),
    (61, 25, 114, 78),
    (61, 26, 99, 70),
    (61, 38, 77, 86),
    (62, 12, 77, 86),
    (62, 14, 102, 129),
    (62, 15, 104, 131),
    (62, 17, 106, 131),
    (62, 18, 104, 110)]

reference = [
    (60, 12, 82, 93),
    (60, 15, 114, 78),
    (60, 21, 116, 96),
    (61, 12, 82, 84),
    (61, 13, 89, 67),
    (61, 17, 97, 70),
    (61, 20, 91, 67),
    (61, 24, 109, 82),
    (61, 25, 110, 78),
    (61, 26, 99, 70),
    (61, 38, 77, 86),
    (62, 12, 77, 86),
    (62, 14, 97, 125),
    (62, 15, 104, 131),
    (62, 17, 106, 131),
    (62, 18, 103, 111)]

geobleu_val = geobleu.calc_geobleu(generated, reference, processes=3)
print("geobleu: {}".format(geobleu_val))

dtw_val = geobleu.calc_dtw(generated, reference, processes=3)
print("dtw: {}".format(dtw_val))

geobleu: 0.21733678721880598
dtw: 5.889002930255253
